<table> <tr>
        <td  style="background-color:#ffffff;">
            <a href="http://qworld.lu.lv" target="_blank"><img src="../images/qworld.jpg" width="50%" align="left"> </a></td>
        <td width="70%" style="background-color:#ffffff;vertical-align:bottom;text-align:right;">
            prepared by Maksim Dimitrijev(<a href="http://qworld.lu.lv/index.php/qlatvia/">QLatvia</a>)
            and Özlem Salehi (<a href="http://qworld.lu.lv/index.php/qturkey/">QTurkey</a>)
        </td>        
</tr></table>

<table width="100%"><tr><td style="color:#bbbbbb;background-color:#ffffff;font-size:11px;font-style:italic;text-align:right;">This cell contains some macros. If there is a problem with displaying mathematical formulas, please run this cell to load these macros. </td></tr></table>
$ \newcommand{\bra}[1]{\langle #1|} $
$ \newcommand{\ket}[1]{|#1\rangle} $
$ \newcommand{\braket}[2]{\langle #1|#2\rangle} $
$ \newcommand{\dot}[2]{ #1 \cdot #2} $
$ \newcommand{\biginner}[2]{\left\langle #1,#2\right\rangle} $
$ \newcommand{\mymatrix}[2]{\left( \begin{array}{#1} #2\end{array} \right)} $
$ \newcommand{\myvector}[1]{\mymatrix{c}{#1}} $
$ \newcommand{\myrvector}[1]{\mymatrix{r}{#1}} $
$ \newcommand{\mypar}[1]{\left( #1 \right)} $
$ \newcommand{\mybigpar}[1]{ \Big( #1 \Big)} $
$ \newcommand{\sqrttwo}{\frac{1}{\sqrt{2}}} $
$ \newcommand{\dsqrttwo}{\dfrac{1}{\sqrt{2}}} $
$ \newcommand{\onehalf}{\frac{1}{2}} $
$ \newcommand{\donehalf}{\dfrac{1}{2}} $
$ \newcommand{\hadamard}{ \mymatrix{rr}{ \sqrttwo & \sqrttwo \\ \sqrttwo & -\sqrttwo }} $
$ \newcommand{\vzero}{\myvector{1\\0}} $
$ \newcommand{\vone}{\myvector{0\\1}} $
$ \newcommand{\stateplus}{\myvector{ \sqrttwo \\  \sqrttwo } } $
$ \newcommand{\stateminus}{ \myrvector{ \sqrttwo \\ -\sqrttwo } } $
$ \newcommand{\myarray}[2]{ \begin{array}{#1}#2\end{array}} $
$ \newcommand{\X}{ \mymatrix{cc}{0 & 1 \\ 1 & 0}  } $
$ \newcommand{\I}{ \mymatrix{rr}{1 & 0 \\ 0 & 1}  } $
$ \newcommand{\Z}{ \mymatrix{rr}{1 & 0 \\ 0 & -1}  } $
$ \newcommand{\Htwo}{ \mymatrix{rrrr}{ \frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & \frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} & \frac{1}{2} } } $
$ \newcommand{\CNOT}{ \mymatrix{cccc}{1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0} } $
$ \newcommand{\norm}[1]{ \left\lVert #1 \right\rVert } $
$ \newcommand{\pstate}[1]{ \lceil \mspace{-1mu} #1 \mspace{-1.5mu} \rfloor } $

<h2>Grover's Search: Implementation</h2>

[Watch Lecture](https://youtu.be/RHR01e7oN9U)

Now we will consider how to implement Grover's search. Let's recall the whole algorithm.

We are given $N=2^n$ elements, and one element is marked. The task is to find this marked element.

We are going to use $n$ qubits. At the beginning we apply Hadamard to each qubit, so we put our quantum state into superposition. The amplitude of each basis state $ \ket{0 \cdots 0}, \ldots, \ket{1 \cdots 1} $ is set to $ \frac{1}{\sqrt{N}} $. After that we iterate the following algorithm for several times:
<ul>
    <li>Make a query: apply a query oracle operator to qubits - it flips the sign of the amplitude of the state that corresponds to the marked element.</li>
    <li>Inversion: apply a diffusion matrix - the amplitude of each state is reflected over the mean of all amplitudes.</li>
</ul>

Let's check how we can implement the query and inversion operations.

<h3>Query operation</h3>

Suppose that there exists a function $f:\{0,1\}^n \rightarrow \{0,1\}$ with the following properties:

\begin{align*}
f(x)&=1 &\mbox{ if $x$ is marked}\\
f(x)&=0 &\mbox{ otherwise}
\end{align*}

Recall that Grover's algorithm does not actually search a list of elements, but given function $f$ with the above properties, it finds the element $x$ such that $f(x)=1$.


<h3>Task 1 (Discussion)</h3>


Consider the following function $f$. Which element is marked ? How many queries do you need to make to $f$ to find out the marked element in the worst case ?
$$
f: \begin{array}{c|c} \mathbf{In} & \mathbf{Out} \\ \hline  00 & 0 \\ 01 & 0 \\ 10 & 0 \\ 11 & 1      \end{array}
$$


<hr>

$f$ is often called as the <font color="blue">oracle</font> or blackbox. Even though $f$ might not be reversible, it can be implemented in a reversible manner by using the following idea.

<img src="../images/foperator.png" width="30%" align="center">

Here $U_f$, the corresponding quantum operator is defined as follows

$$U_f: \ket{x}\ket{y} \mapsto \ket{x}\ket{y \oplus f(x)}, $$

where $\oplus$ denotes bitwise addition modulo 2 (XOR). 


Note that this mapping is reversible. When $\ket{y}=\ket{0}$, you get exactly $f(x)$ in the output qubit.

<h4>Sign flip </h4>

In the rest of the discussion, we will assume that we are given a quantum circuit implementing the operator $U_f$. An operator which flips the sign of the amplitude of the state corresponding to the marked element $x$ can be constructed using <font color="blue">phase-kickback</font>.

Now let's assume that $\ket{y}=\ket{-} = \frac{1}{\sqrt{2}} (\ket{0} - \ket{1})$ and investigate the effect of the operator $U_f$.

\begin{align*}
U_f \ket{x}\ket{-} &= U_f \ket{x}  \frac{1}{\sqrt{2}} \mypar{ \ket{0}-\ket{1} }\\
&= \frac{1}{\sqrt{2}} (U_f\ket{x}\ket{0}-U_f\ket{x}\ket{1}) \\
&= \frac{1}{\sqrt{2}} (\ket{x}\ket{f(x)\oplus 0} - \ket{x}\ket{f(x)\oplus 1}) \\
&= \ket{x} \frac{1}{\sqrt{2}}\mypar{ \ket{f(x)}-\ket{f(x)\oplus 1}  }\\
&= \ket{x} (-1)^{f(x)} \frac{1}{\sqrt{2}} \mypar{ \ket{0}-\ket{1} } ~~~~ (\mbox{This step is tricky})\\
&= (-1)^{f(x)} \ket{x} \ket{-}
\end{align*}

We have the following transformation:
   $$  \ket{x}\ket{-} \xrightarrow{U_f} (-1)^{f(x)} \ket{x}\ket{-} $$

When $f(x)=1$, we see that a phase of -1 is kicked back to the front of the first register. Hence by preparing the output register in state $\ket{-}$ and applying $U_f$, we obtain the sign flip effect.

**Note that the even if we don't know anything about $f$ (that's why it is called a blackbox), we are able to flip the sign of the amplitude of the marked element by making a query to $f$ by setting output qubit to $\ket{-}$.**



<h3>Task 2</h3>


Let $N=4$. Implement the query phase and check the unitary matrix for the query operator. Note that we are interested in the top-left $4 \times 4$ part of the matrix since the remaining parts are due to the ancilla qubit.

You are given a function $f$ and its corresponding quantum operator $U_f$. First run the following cell to load operator $U_f$. Then you can make queries to $f$ by applying the operator $U_f$ via the following command:

```python
Uf(circuit,qreg)
```

In [ ]:
%run ../include/quantum.py

Now use phase kickback to flip the sign of the marked element:

<ul>
    <li>Set output qubit (qreg[2]) to $\ket{-}$ by applying X and H.</li>
    <li>Apply operator $U_f$
    <li>Set output qubit (qreg[2]) back.</li>
</ul>

(Can you guess the marked element by looking at the unitary matrix?)

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer

qreg = QuantumRegister(3)
#No need to define classical register as we are not measuring

mycircuit = QuantumCircuit(qreg)


#
#Your code here
#


job = execute(mycircuit,Aer.get_backend('unitary_simulator'))
u=job.result().get_unitary(mycircuit,decimals=3)

#We are interested in the top-left 4x4 part
for i in range(4):
    s=""
    for j in range(4):
        val = str(u[i][j].real)
        while(len(val)<5): val  = " "+val
        s = s + val
    print(s)
    

mycircuit.draw(output='mpl')

<a href="B90_Grovers_Search_Implementation_Solutions.ipynb#task2">click for our solution</a>

<hr>

In Task 2, we see that the query operator looks like an identity operator with one of the entries being equal to -1, which is responsible for the sign flip. 

$\mymatrix{cccc}{1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & -1}$.

By looking at the matrix, we can guess that the marked element is $\ket{11}$. Note that in general one is not able to peek at the matrix corresponding to the query operator and therefore it is not possible to find out the marked element after a single query. 

<h3>Inversion operator </h3>

[Watch Lecture](https://youtu.be/L9W2f74s7gs)

This is how we implement the inversion operator:

<ul>
    <li>Apply H to all qubits.</li>
    <li>Apply X to all qubits.</li>
    <li>Apply multi-controlled $Z$ operator.</li>
    <li>Apply X to all qubits other than the ancilla.</li>
    <li>Apply H to all qubits other than the ancilla.</li>
</ul>

At this point, let us put a note about multi-controlled $Z$ operator. To implement a multi-controlled $Z$ operator, we use the same idea in multiple controlled constructions to check whether all qubits are in state $\ket{1}$. We store the result in an ancilla qubit, and then apply a $Z$-gate or $CZ$-gate using the ancilla.

In the case of controlled $Z$ operators, we don't need to make a distinction between the control and target qubits.

For example, let's say we apply $CZ$ where the first qubit is the control and the second qubit is the target and our state is $\ket{11}$. After applying $CZ$, we have

$\ket{1} \otimes -\ket{1} = - \ket{11}$.

If we change the control and the target qubits, then we still have the same output:

$-\ket{1} \otimes \ket{1} = - \ket{11}$.

In Qiskit, we implement $CZ$ operator using the following code:

_circuit.cz(qreg[0],qreg[1])_ 


<h4>Why these gates are chosen? (Optional) </h4>

Now let's try to understand why these gates are chosen. Let's recall the inversion operator:

$$ 2 \mymatrix{ccc}{
    \frac{1}{N}  & \cdots & \frac{1}{N} \\ 
    \vdots & \ddots & \vdots \\
    \frac{1}{N}  & \cdots & \frac{1}{N} \\ 
    } 
- I . $$


This operator is also called the <font color="blue"> diffusion operator</font>. 

Recall that the diffusion operator can be expressed as $D=2\ket{u}\bra{u}-I$ where $\ket{u}=H^{\otimes n}\ket{0^n}$ is the equal superposition vector. We will simply denote $\ket{0^n}$ by $\ket{\mathbf{0}}$. 


Instead of $D$, we will implement $-D$. Since the two operators differ by a negative sign, they yield the same observation probabilities. 

For instance, if $D \ket{\psi} = \ket{\phi}$, then $-D \ket{\psi} = -\ket{\phi}$ and the states $\ket{\phi}$ and $-\ket{\phi}$ have identical observation probabilities.

- To start with let's express $-D$ as follows:

\begin{align*}
-D=-(2\ket{u}\bra{u}-I) = I - 2\ket{u}\bra{u} &= I - 2(H^{\otimes n}\ket{\mathbf{0}}\bra{\mathbf{0}}H^{\otimes n}) \\
&= H^{\otimes n}H^{\otimes n} - 2(H^{\otimes n}\ket{\mathbf{0}}\bra{\mathbf{0}}H^{\otimes n})\\
&=H^{\otimes n} (H^{\otimes n} - 2\ket{\mathbf{0}}\bra{\mathbf{0}}H^{\otimes n}) \\
&=H^{\otimes n} (I - 2\ket{\mathbf{0}}\bra{\mathbf{0}})H^{\otimes n}
\end{align*}

<font color="blue"><i>Looking at this expression, we understand why we have $H$ gates at the beginning and at the end</i>

- Now let us look at the effect of applying $I - 2\ket{\mathbf{0}}\bra{\mathbf{0}}$ to any arbitrary state.

$(I - 2\ket{\mathbf{0}}\bra{\mathbf{0}}) \ket{x} = \ket{x} - 2\ket{\mathbf{0}}\braket{\mathbf{0}}{x}.$

If $\ket{x}=\ket{\mathbf{0}}$, since $\braket{\mathbf{0}}{\mathbf{0}}=1$, then $ \ket{x} - 2\ket{\mathbf{0}}\braket{\mathbf{0}}{x} = \ket{\mathbf{0}} - 2\ket{\mathbf{0}}\braket{\mathbf{0}}{\mathbf{0}} =\ket{\mathbf{0}} -2\ket{\mathbf{0}} = - \ket{\mathbf{0}}$.

If $\ket{x}\neq \ket{\mathbf{0}}$, since $\braket{\mathbf{0}}{x}=0$, then $ \ket{x} - 2\ket{\mathbf{0}}\braket{\mathbf{0}}{x} =  \ket{x} - 2\ket{\mathbf{0}}\cdot 0  = \ket{x}$.

Hence, the effect of $I - 2\ket{\mathbf{0}}\bra{\mathbf{0}}$ is flipping the amplitude of the state $\ket{\mathbf{0}}$.

- How to check if the quantum state is equal to $\ket{\bf{0}}$ and flip its amplitude?

We can check if all qubits are set to $\ket{1}$, and then we can implement a multi-controlled $Z$ gate to flip the amplitude.

<font color="blue"> <i> That's why we first apply $X$ gates and then apply multi-controlled $Z$ gate. Afterwards, we set the qubits back by applying $X$ gates again (Recall the multiple controlled constructions notebook.). </i>

<h3>Task 3</h3>

Let $N=4$. Implement the inversion operator and check whether you obtain the following matrix:

$\mymatrix{cccc}{0.5 & -0.5 & -0.5 & -0.5 \\ -0.5 & 0.5 & -0.5 & -0.5 \\ -0.5 & -0.5 & 0.5 & -0.5 \\ -0.5 & -0.5 & -0.5 & 
0.5}$.

In [ ]:
def inversion(circuit,quantum_reg):

#
# your code is here
#

Below you can check the matrix of your inversion operator and how the circuit looks like.

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer

qreg1 =  QuantumRegister(2)

mycircuit1 = QuantumCircuit(qreg1)

inversion(mycircuit1,qreg1)

job = execute(mycircuit1,Aer.get_backend('unitary_simulator'))
u=job.result().get_unitary(mycircuit1,decimals=3)
for i in range(4):
    s=""
    for j in range(4):
        val = str(u[i][j].real)
        while(len(val)<5): val  = " "+val
        s = s + val
    print(s)
    
mycircuit1.draw(output='mpl')

<a href="B90_Grovers_Search_Implementation_Solutions.ipynb#task3">click for our solution</a>

<h3>Task 4: Testing Grover's search</h3>

Now we are ready to test our operations and run Grover's search. Suppose that there are 4 elements in the list and try to find the marked element.

You are given the operator $U_f$. First run the following cell to load it. You can access it via `Uf(circuit,qreg)`.
`qreg[2]` is the ancilla qubit and it is shared by the query and the inversion operators.
    
Which state do you observe the most? 

In [ ]:
%run ..\include\quantum.py

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer

qreg = QuantumRegister(3)
creg = ClassicalRegister(2)

mycircuit = QuantumCircuit(qreg,creg)

#Grover


#initial step - equal superposition
#
#your code here
#

#set ancilla 
#
#your code here
#



#change the number of iterations
iterations=1

#Grover's iterations.
#
#Query operator
#
#Inversion operator
#
   
#set ancilla
#
#your code here
#

mycircuit.measure(qreg[0],creg[0])
mycircuit.measure(qreg[1],creg[1])

job = execute(mycircuit,Aer.get_backend('qasm_simulator'),shots=10000)
counts = job.result().get_counts(mycircuit)

# print the outcome
for outcome in counts:
    print(outcome,"is observed",counts[outcome],"times")

mycircuit.draw(output='mpl')

<a href="B90_Grovers_Search_Implementation_Solutions.ipynb#task4">click for our solution</a>

<h3>Task 5 (Optional, challenging)</h3>

Implement the inversion operation for $n=3$ ($N=8$). This time you will need 4 qubits - 3 for the operation, and one qubit to implement multi-controlled $Z$ gate.


In [ ]:
def big_inversion(circuit,quantum_reg):
    
    
#
# your code is here
#


Below you can check the matrix of your inversion operator. We are interested in the top-left $8 \times 8$ part of the matrix, the remaining parts are because of additional qubits.

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer

big_qreg2 =  QuantumRegister(4)

big_mycircuit2 = QuantumCircuit(big_qreg2)
    
big_inversion(big_mycircuit2,big_qreg2)

job = execute(big_mycircuit2,Aer.get_backend('unitary_simulator'))
u=job.result().get_unitary(big_mycircuit2,decimals=3)
for i in range(8):
    s=""
    for j in range(8):
        val = str(u[i][j].real)
        while(len(val)<6): val  = " "+val
        s = s + val
    print(s)

<a href="B90_Grovers_Search_Implementation_Solutions.ipynb#task5">click for our solution</a>

<h3>Task 6: Testing Grover's search for 8 elements (Optional, challenging)</h3>

Now we will test Grover's search on 8 elements.

You are given the operator $U_{f_8}$. First run the following cell to load it. You can access it via:

```python
Uf_8(circuit,qreg)
```
    
Which state do you observe the most?

In [ ]:
%run ..\include\quantum.py

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer

qreg8 =  QuantumRegister(5)
creg8 = ClassicalRegister(3)

mycircuit8 = QuantumCircuit(qreg8,creg8)

#
#Your code here
#

job = execute(mycircuit8,Aer.get_backend('qasm_simulator'),shots=10000)
counts8 = job.result().get_counts(mycircuit8)
# print the reverse of the outcome
for outcome in counts8:
    print(outcome,"is observed",counts8[outcome],"times")

mycircuit8.draw(output='mpl')

<a href="B90_Grovers_Search_Implementation_Solutions.ipynb#task6">click for our solution</a>

<h3>Task 7 (optional)</h3>

Do you remember the optimal number of iterations to find the marked element with Grover's search? You are given the following code which implements Grover's search. (<i>giant_oracle</i> and <i>giant_diffusion</i> are already implemented.) There is a single marked element. First, determine the total number of elements in the search space. After that, try to find the optimal number of iterations. You can check your guess by changing the value of <i>iteration_count</i>. Please also check less optimal options like <i>iteration_count-1</i>, <i>iteration_count+1</i>.
    
Try also calling <i>giant_oracle2</i> which marks 2 elements and try to find out the optimal number of iterations.

In [ ]:
%run ../include/quantum.py
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer

qreg12 =  QuantumRegister(19)
creg12 = ClassicalRegister(10)

mycircuit12 = QuantumCircuit(qreg12,creg12)

for i in range(10):
    mycircuit12.h(qreg12[i])

mycircuit12.x(qreg12[10])
mycircuit12.h(qreg12[10])

#number of iterations - change this value
iteration_count = 1
for i in range(iteration_count):
    giant_oracle2(mycircuit12,qreg12)
    giant_diffusion(mycircuit12,qreg12)
    
mycircuit12.h(qreg12[10])
mycircuit12.x(qreg12[10])

for i in range(10):
    mycircuit12.measure(qreg12[i],creg12[i])

job = execute(mycircuit12,Aer.get_backend('qasm_simulator'),shots=100000)
counts12 = job.result().get_counts(mycircuit12)
# print the reverse of the outcome
for outcome in counts12:
    print(outcome,"is observed",counts12[outcome],"times")

<h3>Designing oracle functions</h3>

Oracle function $f$ depends on the problem you want to solve. You can model many different problems (such as graph coloring, Traveling Salesman and many more) as a search problem. Elements in your search space correspond to quantum states. Instead of searching the whole space, you design $f$ so that it checks whether an element in the search space is the actual solution and marks it by outputing 1. Then you can use Grover's search to find the solution. 

Now we will design a very simple oracle function, which simply marks one of the elements. Suppose that $N=4$. Our elements in the search space are $\ket{00}$, $\ket{01}$, $\ket{10}$, $\ket{11}$ and suppose that we want to mark the element $\ket{11}$. This means that the function should output 1 if the input qubits are in state 0. This can be simply accomplished with the following function:

In [ ]:
def oracle_11(circuit,qreg):
    circuit1.ccx(qreg[0],qreg[1],qreg[2])
                 

Here qreg[0] and qreg[1] are the input qubits and qreg[2] is the output qubit. Let's check the matrix corresponding to the query phase.

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer

qreg1 =  QuantumRegister(3)

circuit1 = QuantumCircuit(qreg1)

# prepare ancilla qubit
circuit1.x(qreg1[2])
circuit1.h(qreg1[2])

#call the oracle
oracle_11(circuit1,qreg1)

# put ancilla qubit back into state |0>
circuit1.h(qreg1[2])
circuit1.x(qreg1[2])

job = execute(circuit1,Aer.get_backend('unitary_simulator'))
u=job.result().get_unitary(circuit1,decimals=3)
for i in range(4):
    s=""
    for j in range(4):
        val = str(u[i][j].real)
        while(len(val)<5): val  = " "+val
        s = s + val
    print(s)

circuit1.draw(output='mpl')

Let's check another example, now our marked element is $\ket{01}$.

In [ ]:
def oracle_01(circuit,qreg):
    circuit.x(qreg[1])
    circuit.ccx(qreg[0],qreg[1],qreg[2])
    circuit.x(qreg[1])

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer

qreg1 =  QuantumRegister(3)

circuit1 = QuantumCircuit(qreg1)

# prepare ancilla qubit
circuit1.x(qreg1[2])
circuit1.h(qreg1[2])

#call the oracle
oracle_01(circuit1,qreg1)

# put ancilla qubit back into state |0>
circuit1.h(qreg1[2])
circuit1.x(qreg1[2])

job = execute(circuit1,Aer.get_backend('unitary_simulator'))
u=job.result().get_unitary(circuit1,decimals=3)
for i in range(4):
    s=""
    for j in range(4):
        val = str(u[i][j].real)
        while(len(val)<5): val  = " "+val
        s = s + val
    print(s)

circuit1.draw(output='mpl')

<h3>Task 8</h3>

Implement an oracle function which marks the element $\ket{00}$. Run Grover's search with the oracle you have implemented. 

In [ ]:
def oracle_00(circuit,qreg):
    #Your code here
    #
    #

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer

#Grover's search





<a href="B90_Grovers_Search_Implementation_Solutions.ipynb#task8">click for our solution</a>

<h3>Task 9 (Optional)</h3>

Let $N=8$. Implement an oracle function which marks the elements $\ket{001}$ and $\ket{111}$. Run Grover's search with the oracle you have implemented. 

In [ ]:
def oracle_001_111(circuit,qreg):
    #Your code here
    #
    #

Further links:

http://quantumgazette.blogspot.com/2017/12/grovers-algorithm-for-unstructured.html

http://twistedoakstudios.com/blog/Post2644_grovers-quantum-search-algorithm